In [9]:
import logging
from collections import namedtuple
import random

In [10]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [11]:
AUTOMATIC=False
rows=6

In [12]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [13]:
def hardcoded_agent(state: Nim) -> Nimply:
    Evaluator=[0]*4
    for c in state.rows:
        if c==1:
            Evaluator[0]+=1
        if c==2:
            Evaluator[1]+=1
        if c==3:
            Evaluator[2]+=1
        if c>3:
            Evaluator[3]+=1

    if Evaluator[1]+Evaluator[2]+Evaluator[3]==1: #Final rush: aside one row, only ones left in the others
        if Evaluator[0]%2==0:
            for r, c in enumerate(state.rows):
                if c:
                    return Nimply(r,c-1)
        else:
            for r, c in enumerate(state.rows):
                if c:
                    return Nimply(r,c)

    if Evaluator[1]==2 and Evaluator[0]+Evaluator[2]+Evaluator[3]==1: #[0,2,1,0] o [1,2,0,0] o [0,2,0,1] ->[0,2,0,0]
        for r, c in enumerate(state.rows):
                if c and c!=2:
                    return Nimply(r,c)

    if Evaluator[0]==2 and Evaluator[1]==1 and Evaluator[2]==1 and Evaluator[3]==0: #[2,1,1,0]->[1,1,1,0]
        for r, c in enumerate(state.rows):
                if c==1:
                    return Nimply(r,c)

    if Evaluator[0]==1 and Evaluator[1]==1 and Evaluator[2]==2 and Evaluator[3]==0: #[1,1,2,0]->[1,1,1,0]
        for r, c in enumerate(state.rows):
                if c>2:
                    return Nimply(r,c)
    if Evaluator[0]==0 and Evaluator[1]==1 and Evaluator[2]+Evaluator[3]==1: #[0,1,1,0] o [0,1,0,1]->[0,2,0,0]
        for r, c in enumerate(state.rows):
                if c>2:
                    return Nimply(r,c-2)

    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [14]:
def dumb_PCI(state: Nim) -> Nimply:
    """Pick always the minimum(maximum) possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], -m[1])))

In [15]:
def manual(state:Nim)-> Nimply:
    while True:
        try:
            row=int(input("Enter the number of row(from 1 up):"))
            if row<=len(state._rows) and row>0 and state._rows[row-1]!=0:
                row-=1
                break
            else:
                print("No objects in this row. Try again.")
        except ValueError:
            print("Invalid input. Try again.")
    while True:
        try:
            num_objects=int(input("Enter the number of objects to take:"))
            if num_objects>=0 and state._rows[row]-num_objects>=0:
                break
            else:
                print("You cannot take this number of objects, be serious. Try again.")
        except ValueError:
            print("Invalid input. Try again.")
    return Nimply(row,num_objects)

In [16]:
logging.getLogger().setLevel(logging.DEBUG)

if AUTOMATIC:
    strategy = (hardcoded_agent, dumb_PCI)
else:
    strategy = (hardcoded_agent, manual)

nim = Nim(rows)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11>
DEBUG:root:status: After player 0 -> <1 3 5 2 9 11>
DEBUG:root:status: After player 1 -> <1 3 5 2 9 0>
DEBUG:root:status: After player 0 -> <1 3 5 2 1 0>
DEBUG:root:status: After player 1 -> <1 3 0 2 1 0>
DEBUG:root:status: After player 0 -> <0 3 0 2 1 0>
DEBUG:root:status: After player 1 -> <0 2 0 2 1 0>
DEBUG:root:status: After player 0 -> <0 2 0 2 0 0>
DEBUG:root:status: After player 1 -> <0 2 0 2 0 0>
DEBUG:root:status: After player 0 -> <0 1 0 2 0 0>
DEBUG:root:status: After player 1 -> <0 0 0 2 0 0>
DEBUG:root:status: After player 0 -> <0 0 0 1 0 0>


No objects in this row. Try again.
No objects in this row. Try again.
No objects in this row. Try again.
this row has not so many objects. Try again
this row has not so many objects. Try again
this row has not so many objects. Try again


DEBUG:root:status: After player 1 -> <0 0 0 2 0 0>
DEBUG:root:status: After player 0 -> <0 0 0 1 0 0>
DEBUG:root:status: After player 1 -> <0 0 0 0 0 0>
INFO:root:status: Player 0 won!
